In [ ]:
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn import cluster
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib as g
from sklearn.preprocessing import OneHotEncoder
import random

In [ ]:
# Reading the Data

In [ ]:
movie = pd.read_csv("cleaned_movies.csv", encoding='utf-8')
#movie = pd.read_csv("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/cleaned_movies.csv", encoding='utf-8')
movie.head()

# Calculating Genre Correlations

In [ ]:
df = movie
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(ascending=False, kind="quicksort")
genre_corr = so[28::2]
genre_corr[0:28]

In [ ]:
counts = [] #getting the counts per genre
genres = df.columns[2::]
for i in genres:
    counts.append(df[i].value_counts().to_dict())

In [ ]:
#creating counts table
counts_df = pd.DataFrame.from_dict(counts)

counts_df[2] = genres

counts_table = counts_df.drop([0], axis=1)

counts_table = counts_table.rename(columns = {1:'counts', 2:'genre'})
counts_sort = counts_table.sort_values('counts', ascending=False)

#creating column of cumulative sums
sumcol = counts_sort['counts'].sum()
cumsum = counts_sort['counts']/sumcol
counts_sort['cumsum'] = cumsum
counts_sort

# acquiring all column labels we decided we dont want

In [ ]:
garb = counts_sort['genre'].where(counts_sort['cumsum']<.04).tolist()
garb = garb[9::]
garb

In [ ]:
clean_df = df #clean dataset without garbage labels
clean_df = clean_df.drop(garb, axis=1) #removing the unwanted genres from our dataset
clean_df = clean_df.drop(clean_df.columns[0], axis=1)
clean_df.head()

In [ ]:
#New Correlations after removing excess labels
c = clean_df.corr().abs()
s = c.unstack()
so = s.sort_values(ascending=False, kind="quicksort")
genre_corr = so[9::2]
genre_corr[0:12]

In [ ]:
#combining adventure and action
Action_Adventure=[]
for i in range(0, len(clean_df), 1):
    if (clean_df['Action'][i]==1 or clean_df['Adventure'][i]==1):
        Action_Adventure.append(1)
    else:
        Action_Adventure.append(0)
        
#combining Drama and Documentary
Drama_Documentary=[]
for i in range(0, len(clean_df), 1):
    if (clean_df['Drama'][i]==1 or clean_df['Documentary'][i]==1):
        Drama_Documentary.append(1)
    else:
        Drama_Documentary.append(0)

In [ ]:
no_titles = clean_df.drop(["Title"], axis=1) #final_df without the Title columns (final "raw" dataset before undersampling)
clean_df["Action/Adventure"] = Action_Adventure
clean_df["Drama/Documentary"] = Drama_Documentary
final_df = clean_df.drop(['Action', 'Adventure', 'Documentary', 'Drama'], axis=1)
final_df

In [ ]:
#this cell acquires the new cumulative sums after we merging some labels
clean_counts = []
clean_genre = no_titles.columns.tolist()
#clean_genre = final_df.columns.tolist()
for i in clean_genre:
    clean_counts.append(clean_df[i].value_counts().to_dict())
    
c_counts_df = pd.DataFrame.from_dict(clean_counts)
c_counts_df

c_counts_df[2] = clean_genre
c_counts_df
c_counts_table = c_counts_df.drop([0], axis=1)

c_counts_table = c_counts_table.rename(columns = {1:'counts', 2:'genre'})
c_counts_sort = c_counts_table.sort_values('counts', ascending=False)

c_sumcol = c_counts_sort['counts'].sum()
c_cumsum = c_counts_sort['counts']/c_sumcol
c_counts_sort['cumsum'] = c_cumsum
c_counts_sort

# Downsampling!

In [ ]:
sampled = final_df #duplicate our dataset to work with
sampled['sample']=0 #append a column of zeros

In [ ]:
#randomly select 4000 observations from each genre without replacment
random.seed(1)
for i in clean_genre:
    i = np.random.choice(no_titles.where(no_titles[i]==1).index, size=4000, replace=False)
    sampled['sample'][i]=1 #changing the sample value to 1 means we will 'select' this observation
sampled

In [ ]:
undersampled = sampled.loc[sampled['sample']==1] #only keep observations where sample=1
undersampled = undersampled.drop(['sample'], axis=1) #delete sample column, since they will all be 1
undersampled

In [ ]:
undersampled.to_pickle('Undersampled Dataset')

In [ ]:
####################################################################################
##  if we only want to use the top 6 genres without grouping, use this dataframe  ##
####################################################################################
sixgenres = clean_df.drop(['Adventure', 'Documentary', 'Drama'], axis=1)
sixgenres['sample']=0
#randomly select 4000 observations from each genre without replacment
# random.seed(1)
# for i in sixgenres.columns[1::]:
#     i = np.random.choice(sixgenres.where(sixgenres[i]==1).index, size=4000, replace=False)
#     sixgenres['sample'][i]=1 #changing the sample value to 1 means we will 'select' this observation
# sixgenres = sixgenres.loc[sixgenres['sample']==1] #only keep observations where sample=1
# sixgenres = sixgenres.drop(['sample'], axis=1) #delete sample column, since they will all be 1
sixgenres.head()

In [ ]:
genres = sixgenres.columns[1:].tolist()
counts = []
for i in genres:
    counts.append(sixgenres[i].value_counts().to_dict())
    #counts.append((df[i]==1).value_counts().to_dict())
#counts_df = []
counts_df = pd.DataFrame.from_dict(counts)
#counts_df = pd.DataFrame(counts)
#counts_df = pd.DataFrame(counts)
counts_df[2] = genres
#counts_df
counts_table = counts_df.drop([0], axis=1)
#counts_table = counts_table.drop([28])
counts_table = counts_table.rename(columns = {1:'counts', 2:'genre'})
counts_sort = counts_table.sort('counts', ascending=False)
#counts_sort
sumcol = counts_sort['counts'].sum()
cumsum = counts_sort['counts']/sumcol
counts_sort['cumsum'] = cumsum
counts_sort

In [ ]:
col_list = ['Action','Comedy','Crime','Horror','Romance','Thriller']

In [ ]:
sixgenres['Empty'] = sixgenres[col_list].sum(axis = 1)

In [ ]:
df = sixgenres[sixgenres.Empty != 0]

In [ ]:
df.head()

In [ ]:
for i in df.columns[1:]:
    print(i)
    print(df[i].sum())
    

In [ ]:
comedy = df[df['Comedy'] == 1]

In [ ]:
np.random.seed = 1
b = np.random.choice(comedy.Title,size = 4000, replace = False)

In [ ]:
df2 = comedy[comedy['Title'].isin(b)]

In [ ]:
df2.head()

In [ ]:
df2.Title

In [ ]:
df3 = df[(df['Title'].isin(df2.Title) & df['Comedy'] == 1) | (df['Comedy'] == 0)]

In [ ]:
df3.head(50)

In [ ]:
for i in df3.columns[1:]:
    print(i)
    print(df3[i].sum())

In [ ]:
len(df3.Title)

In [ ]:
df4 = df3.drop(df3.columns[-2:],axis= 1)

In [ ]:
df4.head()

In [ ]:
genres = df4.columns[1:].tolist()
counts = []
for i in genres:
    counts.append(df4[i].value_counts().to_dict())
    #counts.append((df[i]==1).value_counts().to_dict())
#counts_df = []
counts_df = pd.DataFrame.from_dict(counts)
#counts_df = pd.DataFrame(counts)
#counts_df = pd.DataFrame(counts)
counts_df[2] = genres
#counts_df
counts_table = counts_df.drop([0], axis=1)
#counts_table = counts_table.drop([28])
counts_table = counts_table.rename(columns = {1:'counts', 2:'genre'})
counts_sort = counts_table.sort('counts', ascending=False)
#counts_sort
sumcol = counts_sort['counts'].sum()
cumsum = counts_sort['counts']/sumcol
counts_sort['cumsum'] = cumsum
counts_sort

In [ ]:
df4.to_pickle('Top6.pkl')